### Own implementation 
### Multiple linear regression
### gradient descent
### Stochastic Gradient Descent
### Forward and Backward Selection 

### Implementation of multiple linear regression by gradient descent


In [711]:
import numpy as np

iteration=30000  

def linear_regression_gd (X,y,learning_rate):
    featureMatrixSizeWithBias=np.size(X,1) #including bias(one's column)
    thetas=np.zeros(featureMatrixSizeWithBias) 
    totalInstance = len(y)  
    costMatrix =[] 

    for i in range(iteration):       
        predictedY=X.dot(thetas) 
        error=predictedY-y        
        thetas = thetas - (learning_rate * (X.T.dot(error) / totalInstance)) # calculate gradient using formula
        costAsMSE = mean_square_error(X, y, thetas)
        costMatrix.append(costAsMSE)
        
    return (thetas ,costMatrix) # return as touples 

def mean_square_error(X,y,theta):
    totalInstance = len(y)
    MSE = np.sum((X.dot(theta) - y)** 2)/(2* totalInstance)
    return MSE

### Extention to implement Stochastic Gradient Descent

In [712]:

def linear_regression_sgd (X,y,learning_rate, batch_size):
    
    trainingDataSize=len(y)
    featureMatrixSizeWithBias=np.size(X,1)  #including bias(one's column)
    thetas=np.zeros(featureMatrixSizeWithBias) 
    costMatrix =[] 
    
    for i in range(iteration):
        shuffledIndexs=np.random.permutation(trainingDataSize) # suffle the training data indexes
        XRandomSet=X[shuffledIndexs]
        yRandomSet=y[shuffledIndexs]
        totalCost=0
        
        for j in range(0,trainingDataSize,batch_size):
            XBatchDataSet= XRandomSet[j:j+batch_size] # get a batch of X
            yBatchDataSet=yRandomSet[j:j+batch_size]  # get a batch of y
            predictedY=XBatchDataSet.dot(thetas)
            error=predictedY-yBatchDataSet
            thetas = thetas - (learning_rate * (XBatchDataSet.T.dot(error) / trainingDataSize)) # calculate gradient using formula
            cost_MSE = mean_square_error(XBatchDataSet,yBatchDataSet, thetas)
            totalCost = totalCost + cost_MSE
        costMatrix.append(totalCost)
                   
    return  (thetas ,costMatrix)     
            

### Loaded  quality of Government Dataset 
from : https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan18.csv.


In [713]:
import pandas as pd
data=pd.read_csv("https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan18.csv")
data=data[["cname","wdi_lifexp","wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"]]
#data.head()

### Correlation of all other variables with the life expectancy (wdi_lifexp)

In [714]:
import numpy as np

def pearsonCorrelationCal(comparingData,comparedWithData):
    correlationCoef={}
    comparedWith=comparedWithData.values
    for columnName in comparingData:
        dataA=comparingData[columnName].values
        if np.issubdtype(dataA.dtype, np.number) == True:
            pearson = np.cov(dataA,comparedWith) [1,0] / (np.std(comparedWith) * np.std(dataA))
            correlationCoef[columnName]=pearson
    return  correlationCoef


comparingData=data.dropna()
comparedWithData=comparingData['wdi_lifexp']

PCC= pearsonCorrelationCal(comparingData,comparedWithData)
print("From custom function ")
PCC

#print("From pandas library ")
#mdata.corr(method='pearson')


From custom function 


{'wdi_lifexp': 1.0087719298245617,
 'wdi_popden': 0.1921300888285124,
 'gle_cgdpc': 0.5023396304773133,
 'bti_acp': 0.42331241614917875,
 'bti_pdi': 0.1993364913893412,
 'fh_pair': 0.5142254232268665,
 'al_ethnic': -0.606459457714409,
 'al_language': -0.6226969196321004,
 'al_religion': -0.316991610514409,
 'bti_aar': 0.1515196264752576,
 'vdem_gender': 0.2027517401678375,
 'bti_ci': -0.4270871781268016,
 'bti_foe': 0.1546978084446338,
 'wdi_araland': 0.024731135841114894,
 'wdi_forest': 0.08866177540349948}

### Applied  GD and SGD
comparision with own implementation with scikit learn libraries
N:B removed all countries with missing values.

In [716]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

dt=data[['wdi_lifexp','wdi_popden','gle_cgdpc']] 
dt=dt.dropna()

X=dt[["wdi_popden","gle_cgdpc"]]
y=dt[["wdi_lifexp"]]

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y).ravel()

#need to add ones matrix in the first column of X as x0=1 was assumed
ones = np.array([np.ones(len(X))]) 
ones=np.reshape(ones,(-1,1)) 
X=np.concatenate((ones,X), axis=1) 


learning_rate=0.01
batch_size=10


thetas ,costMatrix = linear_regression_gd(X,y,learning_rate)
print('With GD \nThetas:')
print(thetas)
print("Initial cost :  %f " % (costMatrix[0]))
print("Final cost :  %f " % (costMatrix[-1]))


print("-----")
thetas ,costMatrix = linear_regression_sgd (X,y,learning_rate, batch_size)
print('With SGD \nThetas:')
print(thetas)
print("Initial cost :  %f " % (costMatrix[0]))
print("Final cost :  %f " % (costMatrix[-1]))

print("-----")
print('with SGDRegressor')
sgd_reg = SGDRegressor(loss='squared_loss',n_iter=iteration, penalty=None, eta0=learning_rate)
sgd_reg.fit(X,y)
print("Thetas:")
print( sgd_reg.coef_)

print("-----")
print('with LinearRegression')
classifier = LinearRegression()
model = classifier.fit(X, y)
y_predict = np.array(classifier.predict(X))
print('Coefficients: ', classifier.coef_)
print('Intercept: ', classifier.intercept_)
print("Mean squared error: %.2f" % mean_squared_error(y, y_predict))


With GD 
Thetas:
[2.89604406e-16 3.94906609e-02 6.24556484e-01]
Initial cost :  0.495727 
Final cost :  0.299000 
-----
With SGD 
Thetas:
[-3.16027071e-05  3.93549347e-02  6.24474021e-01]
Initial cost :  9.416044 
Final cost :  5.582674 
-----
with SGDRegressor


C:\Users\imon_\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Thetas:
[-1.40456750e-04  3.86951092e-02  6.24129276e-01]
-----
with LinearRegression
Coefficients:  [0.         0.03949066 0.62455648]
Intercept:  2.898176307091915e-16
Mean squared error: 0.60


### Regression models to model the life expectancy (wdi_lifexp) in this dataset
Comparision with linear regression, Ridge regression and Lasso using k-fold-cross validation


In [706]:
import seaborn as sb
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

data.head()
dt=data.drop(['cname'], axis=1)
dt=dt.fillna(0)
dt=dt.astype(int)


features=dt.iloc[:,1:15]
target=dt[["wdi_lifexp"]]

scaler = StandardScaler()
features = scaler.fit_transform(features)
target = scaler.fit_transform(target).ravel()

#models
linerRgsnModel = LinearRegression()
ridgeModel = Ridge()
lassoModel = Lasso()

#apply kfold
kfold = model_selection.KFold(n_splits=10, random_state=7)

MSE_LR = model_selection.cross_val_score(linerRgsnModel, features, target, cv=kfold, scoring='neg_mean_squared_error')
MSE_LR=np.sqrt(-MSE_LR).mean()
print("MSE value of LinearRegression :  %f " % (MSE_LR))

MSE_RR = model_selection.cross_val_score(ridgeModel, features, target, cv=kfold, scoring='neg_mean_squared_error')
MSE_RR = np.sqrt(-MSE_RR).mean()
print("MSE value of Ridge Regression :  %f " % (MSE_RR))

MSE_LR = model_selection.cross_val_score(lassoModel, features, target, cv=kfold, scoring='neg_mean_squared_error')
MSE_LR=np.sqrt(-MSE_LR).mean()
print("MSE value of Lasso Regression :  %f " % (MSE_LR))


MSE value of LinearRegression :  0.947424 
MSE value of Ridge Regression :  0.946703 
MSE value of Lasso Regression :  0.950462 


### Forward and Backward Selection algorithms
Applied to the Quality of Government dataset.
Comparision of the results for Forward and Backward selection with each other.

In [719]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

#using Quality of Government dataset from previous
dt=data.drop(['cname'], axis=1)
dt=dt.fillna(0)

X=dt.iloc[:,1:15]
y=dt[["wdi_lifexp"]].values



def forwardSelectionAlgo(X,y): 
    allFeatures=X.columns.tolist()
    remainingFeatures=allFeatures
    includedFeatures=[] # start with empty list
    for i in range(len(allFeatures)):
        pVal = []
        for fcolumn in remainingFeatures:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[includedFeatures+[fcolumn]]))).fit() 
            pVal.append(model.pvalues[fcolumn])    
        minPval = min(pVal) # get the minimum p value    
        if minPval < .01:   # check if the p value is less than .01 then we will take the feature to our model    
            includedFeatures.append(remainingFeatures[pVal.index(minPval)]) # include the feature to our model
            del remainingFeatures[pVal.index(minPval)]
    return includedFeatures    

def backwardSelectionAlgo(X,y): 
    allFeatures=X.columns.tolist()
    remainingFeatures=allFeatures  # start with all features in the list
    for i in range(len(allFeatures)):
        pVal = []
        for fcolumn in remainingFeatures:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[fcolumn]))).fit()
            pVal.append(model.pvalues[fcolumn])    
        maxPval = max(pVal)  # get the maximum p value   
        if maxPval > .05:  # check if the p value is greater than .05 then we will remove the feature from our model       
            #del selectedFeatures[remainingFeatures[pVal.index(maxPval)]] # remove the feature from our model 
            del remainingFeatures[pVal.index(maxPval)] # remove the feature from our model
    return remainingFeatures

print('Before applying algorithm, all features  :')
print(X.columns.tolist()) 

print("____________")
selectedFeaturesByFS = forwardSelectionAlgo(X, y)
print('Selected features from forward selection algorithm :')
print(selectedFeaturesByFS)   

print("____________")
selectedFeaturesByBS = backwardSelectionAlgo(X, y)
print('Selected features from backward selection algorithm :')
print(selectedFeaturesByBS)

Before applying algorithm, all features  :
['wdi_popden', 'gle_cgdpc', 'bti_acp', 'bti_pdi', 'fh_pair', 'al_ethnic', 'al_language', 'al_religion', 'bti_aar', 'vdem_gender', 'bti_ci', 'bti_foe', 'wdi_araland', 'wdi_forest']
____________
Selected features from forward selection algorithm :
['vdem_gender', 'al_language']
____________
Selected features from backward selection algorithm :
['wdi_popden', 'gle_cgdpc', 'al_ethnic', 'al_language', 'vdem_gender', 'wdi_araland']
